# **01 – Data Preparation (Olist Delivery Delays)**

This notebook focuses on loading the raw CSV files, filtering, merging, cleaning and keeping relevant data in a separate CSV file.

This notebook outputs olist_model_data.csv, which will be used in later notebooks for EDA and modelling.

### Step 1: Load raw CSV files

In [2]:
import requests
import os

data_files = [
    'olist_orders_dataset.csv',
    'olist_order_items_dataset.csv',
    'olist_customers_dataset.csv',
    'olist_sellers_dataset.csv',
    'olist_products_dataset.csv'
]

base_url = 'https://raw.githubusercontent.com/aejae-da/bda-olist-project/main/data/'

for file in data_files:
    url = base_url + file
    if not os.path.exists(file):
        print(f"Downloading {file}...")
        response = requests.get(url)
        with open(file, 'wb') as f:
            f.write(response.content)
    print(f"✓ {file} ready")

✓ olist_orders_dataset.csv ready
✓ olist_order_items_dataset.csv ready
✓ olist_customers_dataset.csv ready
✓ olist_sellers_dataset.csv ready
✓ olist_products_dataset.csv ready


In [3]:
import pandas as pd

orders = pd.read_csv(
'olist_orders_dataset.csv',
parse_dates=[
'order_purchase_timestamp',
'order_approved_at',
'order_delivered_carrier_date',
'order_delivered_customer_date',
'order_estimated_delivery_date'
]
)

order_items = pd.read_csv('olist_order_items_dataset.csv')
customers = pd.read_csv('olist_customers_dataset.csv')
sellers = pd.read_csv('olist_sellers_dataset.csv')
products = pd.read_csv('olist_products_dataset.csv')

orders.head()

,order_id,customer_id,order_status,order_purchase_timestamp,order_approved_at,order_delivered_carrier_date,order_delivered_customer_date,order_estimated_delivery_date
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,delivered,2017-10-02 10:56:33,2017-10-02 11:07:15,2017-10-04 19:55:00,2017-10-10 21:25:13,2017-10-18
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,delivered,2018-07-24 20:41:37,2018-07-26 03:24:27,2018-07-26 14:31:00,2018-08-07 15:27:45,2018-08-13
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,delivered,2018-08-08 08:38:49,2018-08-08 08:55:23,2018-08-08 13:50:00,2018-08-17 18:06:29,2018-09-04
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,delivered,2017-11-18 19:28:06,2017-11-18 19:45:59,2017-11-22 13:39:59,2017-12-02 00:28:42,2017-12-15
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,delivered,2018-02-13 21:18:39,2018-02-13 22:20:29,2018-02-14 19:46:34,2018-02-16 18:17:02,2018-02-26


### Step 2: Filter delivered orders, create delay variables

Filtering the orders and creating variables would help in defining the target and giving us the result (Target: late_delivery_flag (1 = delivered after estimate, 0 = on time/early).

In [4]:
#Keep only delivered orders with known estimated date
orders_clean = orders.dropna(
    subset=['order_delivered_customer_date','order_estimated_delivery_date']
).copy()

#Calculate delivery time (purchase to actual delivery) in days
orders_clean['delivery_time_days'] = (
orders_clean['order_delivered_customer_date'] - orders_clean['order_purchase_timestamp']
).dt.days

#Calculate delay in days (actual - estimated)
orders_clean['delay_days'] = (
orders_clean['order_delivered_customer_date'] - orders_clean['order_estimated_delivery_date']
).dt.days

#Create binary target: 1 = late (delivered after estimate), 0 = on time or early
orders_clean['late_delivery_flag'] = (orders_clean['delay_days'] > 0).astype(int)

orders_clean[['delivery_time_days', 'delay_days', 'late_delivery_flag']].describe()

,delivery_time_days,delay_days,late_delivery_flag
count,96476.000000,96476.000000,96476.000000
mean,12.094086,-11.876881,0.067737
std,9.551746,10.183854,0.251295
min,0.000000,-147.000000,0.000000
25%,6.000000,-17.000000,0.000000
50%,10.000000,-12.000000,0.000000
75%,15.000000,-7.000000,0.000000
max,209.000000,188.000000,1.000000


In [5]:
orders_clean[['delivery_time_days', 'delay_days']].describe()

orders_clean['late_delivery_flag'].value_counts(normalize=True)

,proportion
late_delivery_flag,
0,0.932263
1,0.067737


### Step 3: Merge orders with items, customers, sellers, products

In [6]:
#Merge orders with order_items
df = orders_clean.merge(order_items, on='order_id', how='left')

#Merge with customers
df = df.merge(customers, on='customer_id', how='left')

#Merge with sellers
df = df.merge(sellers, on='seller_id', how='left')

#Merge with products
df = df.merge(products, on='product_id', how='left')

df.head()
df.shape

print("Merged df shape:", df.shape)

Merged df shape: (110196, 32)


### Step 4: Keep relevant columns and basic cleaning

In [7]:
cols_to_keep = [
# IDs
'order_id', 'customer_id', 'customer_unique_id', 'seller_id', 'product_id',

# Time fields
'order_purchase_timestamp',
'order_delivered_customer_date',
'order_estimated_delivery_date',
'delivery_time_days',
'delay_days',
'late_delivery_flag',

# Monetary and product info
'price', 'freight_value',
'product_category_name',

# Customer and seller locations
'customer_city', 'customer_state',
'seller_city', 'seller_state'
]

df = df[cols_to_keep].copy()
df.head()

,order_id,customer_id,customer_unique_id,seller_id,product_id,order_purchase_timestamp,order_delivered_customer_date,order_estimated_delivery_date,delivery_time_days,delay_days,late_delivery_flag,price,freight_value,product_category_name,customer_city,customer_state,seller_city,seller_state
0,e481f51cbdc54678b7cc49136f2d6af7,9ef432eb6251297304e76186b10a928d,7c396fd4830fd04220f754e42b4e5bff,3504c0cb71d7fa48d967e0e4c94d59d9,87285b34884572647811a353c7ac498a,2017-10-02 10:56:33,2017-10-10 21:25:13,2017-10-18,8,-8,0,29.99,8.72,utilidades_domesticas,sao paulo,SP,maua,SP
1,53cdb2fc8bc7dce0b6741e2150273451,b0830fb4747a6c6d20dea0b8c802d7ef,af07308b275d755c9edb36a90c618231,289cdb325fb7e7f891c38608bf9e0962,595fac2a385ac33a80bd5114aec74eb8,2018-07-24 20:41:37,2018-08-07 15:27:45,2018-08-13,13,-6,0,118.70,22.76,perfumaria,barreiras,BA,belo horizonte,SP
2,47770eb9100c2d0c44946d9cf07ec65d,41ce2a54c0b03bf3443c3d931a367089,3a653a41f6f9fc3d2a113cf8398680e8,4869f7a5dfa277a7dca6462dcf3b52b2,aa4383b373c6aca5d8797843e5594415,2018-08-08 08:38:49,2018-08-17 18:06:29,2018-09-04,9,-18,0,159.90,19.22,automotivo,vianopolis,GO,guariba,SP
3,949d5b44dbf5de918fe9c16f97b45f8a,f88197465ea7920adcdbec7375364d82,7c142cf63193a1473d2e66489a9ae977,66922902710d126a0e7d26b0e3805106,d0b61bfb1de832b15ba9d266ca96e5b0,2017-11-18 19:28:06,2017-12-02 00:28:42,2017-12-15,13,-13,0,45.00,27.20,pet_shop,sao goncalo do amarante,RN,belo horizonte,MG
4,ad21c59c0840e6cb83a9ceb5573f8159,8ab97904e6daea8866dbdbc4fb7aad2c,72632f0f9dd73dfee390c9b22eb56dd6,2c9e548be18521d1c43cde1c582c6de8,65266b2da20d04dbe00c5c2d3bb7859e,2018-02-13 21:18:39,2018-02-16 18:17:02,2018-02-26,2,-10,0,19.90,8.72,papelaria,santo andre,SP,mogi das cruzes,SP


Basic cleaning: remove extreme outliers

To ensure that very unusual records do not distort the analysis, simple threshold filters are applied to the delivery-related variables. These cutoffs are somewhat arbitrary but chosen to be reasonable based on typical delivery patterns (for example, removing negative or extremely long delivery times and unrealistic delay values). It is not intended to capture each and every unusual observation in the process of building the model itself.

In [8]:
df = df[df['delivery_time_days'] >= 0]
df = df[df['delivery_time_days'] <= 60] # arbitrary upper limit
df = df[df['delay_days'] >= -20] # allow early deliveries but not crazy
df = df[df['delay_days'] <= 40]

df['late_delivery_flag'].value_counts(normalize=True)

,proportion
late_delivery_flag,
0,0.926029
1,0.073971


### Step 5: Save olist_model_data.csv

In [9]:
# Save uncompressed for local work
df.to_csv('olist_model_data.csv', index=False)

In [11]:
# Compress for GitHub upload (25MB limit workaround)
import gzip
import shutil
with open('olist_model_data.csv', 'rb') as f_in:
    with gzip.open('olist_model_data.csv.gz', 'wb') as f_out:
        shutil.copyfileobj(f_in, f_out)

print("Saved both:")
print("- olist_model_data.csv (uncompressed, for local work)")
print("- olist_model_data.csv.gz (compressed, for GitHub upload)")

#Download Original uncompressed file for local work
from google.colab import files
files.download('olist_model_data.csv')

#Download Compressed file for github
from google.colab import files
files.download('olist_model_data.csv.gz')


Saved both:
- olist_model_data.csv (uncompressed, for local work)
- olist_model_data.csv.gz (compressed, for GitHub upload)


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>